<a href="https://colab.research.google.com/github/Shahabshms/Numerical_Methods_for_ML_and_AI_Solution_4/blob/main/_4301_HW4_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#2 
In Python, write a function that takes as input the matrix $A$ and an integer $k > 0$ and returns a matrix $C$ and $U$ obtained by running block coordinate descent to convergence from
a random starting point.

As we metntioned in part 1, the objective function is
\begin{align*}
f(C,U) &= \sum_{i=1}^m\sum_{j=1}^n \left(A_{i,j}\log(\frac{A_{i,j}}{\sum_{k=1}^K C_{i,k}U_{k,j}}) - A_{i,j} + \sum_{k=1}^K C_{i,k}U_{k,j}\right).
\end{align*}

As usual, for implementing gradient descent we need gradient. So, for $a\in\{1,\dots,m\}$ and $b\in\{1,\dots,n\}$ we have
\begin{align*}
\frac{\partial f(C,U)}{\partial C_{a,b}} &= \sum_{j=1}^n \left(-A_{a,j}\frac{U_{b,j}}{\sum_{k=1}^K C_{a,k}U_{k,j}} + U_{b,j}\right).
\end{align*}

In [ ]:
def get_gradient_wrt_C(C,U):
  gradient_C = np.zeros(C.shape)
  for a in range(C.shape[0]):
    for b in range(C.shape[1]):
      gradient_C[a,b] = get_derivative_wrt_C(C,U,a,b)
  return gradient_C

def get_derivative_wrt_C(C,U,a,b):
  derivative = 0
  for j in range(U.shape[1]):
    denum = 0
    for k in range(U.shape[0]):
      denum += C[a,k]*U[k,j]
    
    if denum == 0: # ?????? 
      return 0
      
    derivative += -A[a,j] * (U[b,j] / denum ) + U[b,j]

  return derivative

(If you pay attention you see that there is a small trick in the implemenation. Since it is possible to have $(CU)_{a,b} = 0$, the fraction $\frac{A_{a,b}}{(CU)_{a,b}}$ may not be a number. So, if we encounter $(CU)_{a,b} = 0$, we postpone updating that particlar entry to the next iteration. Or in other words, we put the derivative equal to zero.)

And
\begin{align*}
\frac{\partial f(C,U)}{\partial U_{a,b}} = \sum_{i=1}^m \left(-A_{i,b}\frac{C_{i,a}}{\sum_{k=1}^K C_{i,k}U_{k,b}} + C_{i,a}\right).
\end{align*}

In [ ]:
def get_gradient_wrt_U(C,U):
  gradient_U = np.zeros(U.shape)
  for a in range(U.shape[0]):
    for b in range(U.shape[1]):
      gradient_U[a,b] = get_derivative_wrt_U(C,U,a,b)
  return gradient_U

def get_derivative_wrt_U(C,U,a,b):
  derivative = 0
  for i in range(C.shape[0]):
    denum = 0
    for k in range(C.shape[1]):
      denum += C[i,k]*U[k,b]

    if denum == 0: # ??????
      return 0

    derivative += -A[i,b] * (C[i,a] / denum ) + C[i,a]

  return derivative

In [ ]:
def projected_gradient_descent(initial_C,initial_U,max_iterations):
  C = initial_C
  U = initial_U

  for iteration in range(max_iterations):
    nu = 0.01 # Learning Rate
    
    gradient_wrt_C = get_gradient_wrt_C(C,U)
    C = C - nu*gradient_wrt_C # Update C
    C[C<0] = 0 # projection

    gradient_wrt_U = get_gradient_wrt_U(C,U) 
    U = U - nu*gradient_wrt_U # Update U
    U[U<0] = 0 # projection

    if np.max(abs(A-C@U)) < np.max(A)/50:
      break

  return C,U

Try to compare the impact of large $K$ and small $K$.
Also, try different learning rates. 

In [ ]:
import numpy as np

m = 5
n = 4
K = 2
A = np.random.randint(50,size = (m,n))

##################################################################################
# m=5  #To see that this algorithm may not converge, look at this example.
# n=5
# K=2
# A = np.matrix([[ 6, 47, 49, 40, 27],
#                [12, 48,  6, 42, 33],
#                [27, 23, 33,  1,  4],
#                [17, 49, 48, 27,  2],
#                [30, 34, 27, 13, 41]])
##################################################################################

print('A: \n',A)
for count in range(5):
  C = np.random.randint(10,size = (m,K))
  U = np.random.randint(10,size = (K,n))

  #print("initial C:\n",C)
  #print("initial U:\n",U)

  max_iterations = 10000

  C,U = projected_gradient_descent(C,U,max_iterations)

  print("Final B:\n",np.round((C@U)))
  #print('|A-B|:\n',np.round(abs(A - C@U)))
  print('------')

A: 
 [[ 3 27 15  6]
 [40 33  4 28]
 [47 32 25 31]
 [24 39 23  7]
 [ 6 31 42 36]]
Final B:
 [[ 3. 18. 19. 11.]
 [47. 30.  5. 23.]
 [42. 42. 21. 29.]
 [21. 30. 21. 20.]
 [ 6. 41. 43. 25.]]
------
Final B:
 [[ 5. 19. 20.  6.]
 [33. 39.  4. 29.]
 [34. 51. 20. 31.]
 [39. 43.  0. 34.]
 [ 9. 10.  0.  8.]]
------
Final B:
 [[ 3. 18. 19. 11.]
 [47. 30.  5. 23.]
 [42. 42. 21. 29.]
 [21. 30. 21. 20.]
 [ 6. 41. 43. 25.]]
------
Final B:
 [[ 3. 18. 19. 11.]
 [47. 30.  5. 23.]
 [42. 42. 21. 29.]
 [21. 30. 21. 20.]
 [ 6. 41. 43. 25.]]
------
Final B:
 [[ 3. 18. 19. 11.]
 [47. 30.  5. 23.]
 [42. 42. 21. 29.]
 [21. 30. 21. 20.]
 [ 6. 41. 43. 25.]]
------
